In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import sampler,TensorDataset
import torchvision.datasets as dset
import torchvision.transforms as T
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
dtype = torch.float32
print("CUDA: ",torch.cuda.is_available())
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
#DataLoader
class Data(dataset.Dataset):
def __init__(self):
    super(Data,self).__init__()
    pass

def __getitem__(self):
    pass

def __len__(self):
    pass


In [ ]:
train_data = pd.read_csv("/home/cth/code/kaggle/train.csv")
train_label = train_data['label'].values
train_images = (train_data.iloc[:,1:].values/255.0).astype("float32")
train_images = train_images.reshape(train_images.shape[0],1,28,28)
print(train_images.shape)
train_images,val_images,train_label,val_label = train_test_split(train_images,train_label,random_state=123,stratify=train_label)
print(train_images.shape)
print(train_label.shape)

In [ ]:
for i in range(4):
    plt.subplot(2,2,i+1)
    plt.imshow(train_images[i].squeeze(),cmap=plt.get_cmap("gray"))
    plt.title(train_label[i])

In [ ]:
train_images = torch.tensor(train_images)
train_label = torch.tensor(train_label)
train_tensor = TensorDataset(train_images,train_label)

val_images = torch.tensor(val_images)
val_label = torch.tensor(val_label)
val_tensor = TensorDataset(val_images,val_label)

In [ ]:
batch_size =  100
print_every = 100
loader_train = DataLoader(train_tensor,batch_size=batch_size,num_workers=4)
loader_val = DataLoader(val_tensor,batch_size=batch_size)

In [ ]:
def check_accuracy_part34(loader, model):
    # if loader.dataset.train:
    #     print('Checking accuracy on validation set')
    # else:
    #     print('Checking accuracy on test set')   
    num_correct = 0
    num_samples = 0
    model.eval()  # set model to evaluation mode
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)
            scores = model(x)
            _, preds = scores.max(1)
            num_correct += (preds == y).sum()
            num_samples += preds.size(0)
        acc = float(num_correct) / num_samples
        print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))

def train_part34(model, optimizer, epochs=1):
    """
    Train a model on CIFAR-10 using the PyTorch Module API.
    
    Inputs:
    - model: A PyTorch Module giving the model to train.
    - optimizer: An Optimizer object we will use to train the model
    - epochs: (Optional) A Python integer giving the number of epochs to train for
    
    Returns: Nothing, but prints model accuracies during training.
    """
    model = model.to(device=device)  # move the model parameters to CPU/GPU
    for e in range(epochs):
        for t, (x, y) in enumerate(loader_train):
            model.train()  # put model to training mode
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)

            scores = model(x)
            loss = F.cross_entropy(scores, y)

            # Zero out all of the gradients for the variables which the optimizer
            # will update.
            optimizer.zero_grad()

            # This is the backwards pass: compute the gradient of the loss with
            # respect to each  parameter of the model.
            loss.backward()

            # Actually update the parameters of the model using the gradients
            # computed by the backwards pass.
            optimizer.step()

            if t % print_every == 0:
                print('Iteration %d, loss = %.4f' % (t, loss.item()))
                check_accuracy_part34(loader_val, model)
                print()
                

In [ ]:
class BottleNeck(nn.Module):
    """
    接收 inchannel 道的输入
    输出 4*midchannel 道的输出
    """
    def __init__(self,inchannel,midchannel,stride=1):
        super(BottleNeck,self).__init__()
        self.inchannel = inchannel
        self.midchannel = midchannel
        self.stride = stride

        self.conv1 = nn.Conv2d(self.inchannel,self.midchannel,kernel_size=1,stride=1,bias=False)
        self.bn1 = nn.BatchNorm2d(self.midchannel)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(self.midchannel,self.midchannel,kernel_size=3,stride=stride,padding=1,bias=False)
        self.bn2 = nn.BatchNorm2d(self.midchannel)
        self.conv3 = nn.Conv2d(self.midchannel,4*self.midchannel,kernel_size=1,stride=1,bias=False)
        self.bn3 = nn.BatchNorm2d(4*self.midchannel)
        self.downsample = nn.Sequential(
            nn.Conv2d(self.inchannel,4*self.midchannel,kernel_size=1,stride=1,bias=False),
            nn.BatchNorm2d(4*self.midchannel)
        )
        if self.stride != 1:
            self.downsample = nn.Sequential(
                nn.Conv2d(self.inchannel,4*self.midchannel,kernel_size=1,stride=1,bias=False),
                nn.Conv2d(4*self.midchannel,4*self.midchannel,1,self.stride,bias=False),
                nn.BatchNorm2d(4*self.midchannel)
            )

    def forward(self,x):
        identity = x
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.conv3(x)
        x = self.bn3(x)

        identity = self.downsample(identity)

        x += identity
        x = self.relu(x)
        return x


class Flatten(nn.Module):
    def forward(self,x):
        return x.reshape(x.shape[0],-1)


model = nn.Sequential(
    nn.Conv2d(1,256,1,1,bias=False),
    nn.BatchNorm2d(256),
    nn.ReLU(),

    BottleNeck(256,128),
    BottleNeck(128*4,128),
    BottleNeck(128*4,128),
    BottleNeck(128*4,128),

    BottleNeck(128*4,256,2),
    BottleNeck(256*4,256),
    BottleNeck(256*4,256),
    BottleNeck(256*4,256),
    BottleNeck(256*4,256),
    BottleNeck(256*4,256),

    BottleNeck(256*4,512,2),
    BottleNeck(512*4,512),
    BottleNeck(512*4,512),

    #2048 7 7
    nn.AvgPool2d(2,2,padding=1),
    Flatten(),
    nn.Linear(2048*4*4,10)
)

data = next(enumerate(loader_train))[-1][0].type(dtype)
print(data.shape)
b = model
out = b(data)
print(out.size())

In [ ]:
optim = torch.optim.SGD(model.parameters(),lr=0.001,momentum=0.9,nesterov=True)
train_part34(model,optim,10)

In [ ]:
class Block(nn.Module):
    """
    两层3*3卷积核的基本组件\n
    conv -> bn -> relu -> conv -> bn ->relu(bn+x)
    """
    def __init__(self,inchannel,outchannel,stride=1):
        super(Block,self).__init__()
        #第一个卷积层要做降维
        self.inchannel = inchannel
        self.outchannel = outchannel
        self.stride = stride

        self.conv1 = nn.Conv2d(inchannel,outchannel,kernel_size=3,stride=stride,padding=1,bias=False)
        self.bn1 = nn.BatchNorm2d(outchannel)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(outchannel,outchannel,kernel_size=3,stride=1,padding=1,bias=False)
        self.bn2 = nn.BatchNorm2d(outchannel)
        self.downsampler = nn.Sequential()
        if self.stride!=1 or self.inchannel!=self.outchannel:
            self.downsampler = nn.Sequential(
                nn.Conv2d(self.inchannel,self.outchannel,1,self.stride,bias=False),
                nn.BatchNorm2d(self.outchannel)
            )
    
    def forward(self,x):
        identity = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)

        if self.stride!=1 or self.inchannel!=self.outchannel:
            identity = self.downsampler(identity)
        
        out += identity
        out = self.relu(out)
        return out


#torch.Size([64, 3, 32, 32])
class Flatten(nn.Module):
    def forward(self,x):
        return x.reshape(x.shape[0],-1)

model = nn.Sequential(

    #conv1.x
    nn.Conv2d(1,128,3,1,1,bias=False),
    nn.BatchNorm2d(128),
    nn.ReLU(),

    #conv2.x
    Block(64,64),
    Block(64,64),

    # #conv3.x
    # Block(128,128),
    # Block(128,128),
    # Block(128,128),
    # Block(128,128),

    # Block(128,256,2),
    # Block(256,256),
    # Block(256,256),
    # Block(256,256),
    # Block(256,256),
    # Block(256,256),

    # Block(256,512,2),
    # Block(512,512),
    # Block(512,512),

    # nn.AvgPool2d(4),
    # Flatten(),
    # nn.Linear(512,10)
)
data = next(enumerate(loader_train))[-1][0].type(dtype)
out = model(data)
print(out.size())

In [ ]:
optim = torch.optim.SGD(model.parameters(),lr=0.001,momentum=0.9,nesterov=True)
train_part34(model,optim,10)

In [ ]:
print(model)

In [ ]:
# train_data = pd.read_csv("/home/cth/code/kaggle/train.csv")
# train_label = train_data['label'].values
# train_images = (train_data.iloc[:,1:].values/255.0).astype("float32")
# train_images = train_images.reshape(train_images.shape[0],1,28,28)

test_data = pd.read_csv("/home/cth/code/kaggle/test.csv")
test_data = test_data.values.astype("float32")/255.0
res = []
test_data = torch.tensor(test_data.reshape(test_data.shape[0],1,28,28))
test_data = test_data.to(device)
for i in range(test_data.shape[0]):
    now = test_data[i].reshape(1,1,28,28)
    score = model(now)
    res.append(score.argmax())

In [ ]:
test_data = pd.read_csv("/home/cth/code/kaggle/test.csv")
test_data = test_data.values.astype("float32")/255.0
test_data = test_data.reshape(test_data.shape[0],1,28,28)
test_data = torch.tensor(test_data)
loader_text = DataLoader(test_data)
model.eval()
res = []
for idx,x in enumerate(loader_text):
    x = x.to(device)
    scores = model(x)
    _,preds = scores.max(1)
    res.append(preds.item())
print(res)

In [ ]:
res[3] = 0
for i in range(16):
    plt.subplot(4,4,i+1)
    plt.imshow(test_data[i].squeeze().reshape(28,28))
    plt.title(res[i])

In [ ]:
example = pd.read_csv("/home/cth/code/kaggle/sample_submission.csv")
example['Label'] = res
example.head()

In [ ]:
example.to_csv("/home/cth/code/toys/submission.csv",index=False)

In [ ]:
res = [i.item() for i in res]

In [ ]:
res

In [ ]:
samp = pd.read_csv("/home/cth/code/kaggle/sample_submission.csv")
samp.iloc[:,1] = res

In [ ]:
samp.to_csv("submission.csv",index=False)